### <p style="font-family:Garamond,serif">This notebook is for scraping tweets. </p> 

In [5]:
#Install the twitter scraping library
!pip install snscrape

In [2]:
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction import _stop_words
from sklearn.metrics.pairwise import cosine_similarity as cosine
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.tokenize.casual import casual_tokenize
import pandas as pd
from nlpia.data.loaders import get_data
from sklearn.decomposition import TruncatedSVD
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/pugnlp/constants.py:136: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  [datetime.datetime, pd.datetime, pd.Timestamp])
/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/pugnlp/constants.py:158: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  MIN_TIMESTAMP = pd.Timestamp(pd.datetime(1677, 9, 22, 0, 12, 44), tz='utc')
/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/pugnlp/tutil.py:100: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  np = pd.np
/Users/chenxifang/opt/anaconda3/lib/python3.8/site-packages/pugnlp/util.py:80: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Im

### <p style="font-family:Garamond,serif"> These code are from NLP course's notebook 4.1 Task. I'm using the sns scrape to scrape data from Twitter. </p>

In [3]:
import snscrape.modules.twitter as sntwitter

In [4]:
#Initialise your data frame
hashtag_pfizer_df = pd.DataFrame(columns = ["text", "label"])
hashtag_pfizer_df.shape

(0, 2)

In [5]:
#How many to grab
maxTweets = 10000

###Comment out methods below as appropriate 

# ---EITHER--- Keyword search
# keyword = 'data'
# data = sntwitter.TwitterSearchScraper(keyword + '-filter:links -filter:replies').get_items()

# ---OR--- User search
#user = 'ASTRAZENECAUK'
#data = sntwitter.TwitterSearchScraper('from:ASTRAZENECAUK').get_items()

# ---OR--- Hastag search
hashtag = '#pfizer'
data = sntwitter.TwitterHashtagScraper(hashtag + '-filter:links -filter:replies').get_items()

#Set the label for this group
hashtag_pfizer = []
#Collect in array and pair each tweet with a class label
for i,tweet in enumerate(data):
    if i >= maxTweets:
        break   
    hashtag_pfizer = hashtag_pfizer + [[tweet.content]]


In [6]:
len(hashtag_pfizer)
str(hashtag_pfizer[1])
print(type(str(hashtag_pfizer[0]))) #make sure that the data type in the array are stings
str(hashtag_pfizer[22]) #Check the tweets in the array

<class 'str'>


"['Leggi le tendenze e capisci che la maggior parte degli italiani non capisce quello che sta accadendo. #mascherine #Covid #Omicron #Pfizer #Russia #Ucraina #Kharkiv #ZelenskyWarCriminal']"

### <p style="font-family:Garamond,serif">I initially tried to use TextBlob to filter out English Tweets, however, the function has limitation on the amount of input data per-day, so I had to give up on this function.</p>  

In [208]:
#from textblob import TextBlob, Word, Blobber

In [7]:
!pip install -U pip setuptools wheel
!pip install -U spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 15.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.5 MB/s eta 0:00:00a 0:00:01
  Using cached wheel-0.37.1-py2.py3-none-any.whl (35 kB)
  Attempting uninstall: wheel
    Found existing installation: wheel 0.37.0
    Uninstalling wheel-0.37.0:
      Successfully uninstalled wheel-0.37.0
  Attempting uninstall: setuptools
    Found existing installation: setuptools 59.2.0
    Uninstalling setuptools-59.2.0:
      Successfully uninstalled setuptools-59.2.0
  Attempting uninstall: pip
    Found existing installation: pip 22.1.1
    Uninstalling pip-22.1.1:
      Successfully uninstalled pip-22.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 4.2.5 requires pyqt5<5.13, which is not installed.
spyder 4.2.5 requires pyqtwebengine<5.13, which is not installed.
co

In [8]:
!python -m spacy download en_core_web_sm
!pip install spacy-langdetect #intall spacy language detect to filer language

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 15.9 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 3.2.0
    Uninstalling en-core-web-sm-3.2.0:
      Successfully uninstalled en-core-web-sm-3.2.0
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [11]:
en_hashtag_pfizer = [] #create an array for storing English Tweets only

In [12]:
#import spacy for language detection
#code sourse : https://stackoverflow.com/questions/66712753/how-to-use-languagedetector-from-spacy-langdetect-package
import spacy
from spacy.language import Language
from spacy_langdetect import LanguageDetector

In [14]:
#This bit of code only need to run once !
def get_lang_detector(nlp, name):
    return LanguageDetector()
nlp = spacy.load("en_core_web_sm")
Language.factory("language_detector", func=get_lang_detector)
nlp.add_pipe('language_detector', last=True)

RegistryError: [E893] Could not find function 'spacy.Tagger.v2' in function registry 'architectures'. If you're using a custom function, make sure the code is available. If the function is provided by a third-party package, e.g. spacy-transformers, make sure the package is installed in your environment.

Available names: spacy-legacy.CharacterEmbed.v1, spacy-legacy.HashEmbedCNN.v1, spacy-legacy.MaxoutWindowEncoder.v1, spacy-legacy.MishWindowEncoder.v1, spacy-legacy.MultiHashEmbed.v1, spacy-legacy.TextCatBOW.v1, spacy-legacy.TextCatCNN.v1, spacy-legacy.TextCatEnsemble.v1, spacy-legacy.Tok2Vec.v1, spacy-legacy.TransitionBasedParser.v1, spacy.CharacterEmbed.v2, spacy.EntityLinker.v1, spacy.HashEmbedCNN.v2, spacy.MaxoutWindowEncoder.v2, spacy.MishWindowEncoder.v2, spacy.MultiHashEmbed.v2, spacy.PretrainCharacters.v1, spacy.PretrainVectors.v1, spacy.SpanCategorizer.v1, spacy.Tagger.v1, spacy.TextCatBOW.v2, spacy.TextCatCNN.v2, spacy.TextCatEnsemble.v2, spacy.TextCatLowData.v1, spacy.Tok2Vec

In [33]:
text = str(hashtag_pfizer[2])
doc = nlp(text)
print(doc._.language)
type(doc._.language)

#turn dict into array 
lang_doc = list(dict.items(doc._.language))
lang_array = np.array(lang_doc)

print(lang_array)

{'language': 'pt', 'score': 0.9999961485552193}
[['language' 'pt']
 ['score' '0.9999955735145803']]


In [36]:
for i in range(100000):
    text = str(hashtag_pfizer[i])
    doc = nlp(text)
    doc._.language
    lang_doc = list(dict.items(doc._.language))
    lang_array = np.array(lang_doc)
    #print(lang_array)
    if lang_array[0][1] == 'en':
        en_hashtag_pfizer.append(text)
#print(en_lang)  


(80104, 1)

In [ ]:
en_hashtag_pfizer[50]

In [96]:
en_hashtag_pfizer_df = pd.DataFrame(en_hashtag_pfizer, columns = ["text"])
en_hashtag_pfizer_df.shape


(80104, 1)

In [97]:
#>>>For cleaning dataframe if store wrong<<<
#en_hashtag_pfizer_df.drop(en_hashtag_pfizer_df.index, inplace=True)
#df_en_lang.drop(df_en_lang.loc[:80103].columns, axis = 1)

In [100]:
!pip install langdetect

     |████████████████████████████████| 981 kB 6.5 MB/s eta 0:00:01
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993221 sha256=f3438c88683b2ad9e93ec1a9b22a6fdb99c43247b419a2a8258683be2e2cb08e
  Stored in directory: /Users/chenxifang/Library/Caches/pip/wheels/13/c7/b0/79f66658626032e78fc1a83103690ef6797d551cb22e56e734
Successfully built langdetect


In [98]:
en_hashtag_pfizer_df

,text
0,['I got my #Pfizer booster shot today and the ...
1,"['If the U.S. would release the vaccine, deman..."
2,"[""Happy to get my #Pfizer booster on Saturday ..."
3,['Is the U.S. Treasury heavy in #Pfizer which ...
4,['They can always reverse it at a later date. ...
...,...
80099,['The two #Pfizer scientists responsible for t...
80100,['Stock markets: the big fat #Coronarotation\n...
80101,"['As I have long said, #Pfizer and the others ..."
80102,['Sure is interesting timing that the announce...


In [16]:
#Write to file
file_name = "en_hashtag_pfizer"
en_hashtag_pfizer_df.to_csv("data/" + file_name + ".tsv")

NameError: name 'en_hashtag_pfizer_df' is not defined

In [18]:
#Read back in
file_name = "en_hashtag_pfizer"
en_hashtag_pfizer= pd.read_csv("data/" + file_name + ".tsv")

FileNotFoundError: [Errno 2] No such file or directory: 'data/en_hashtag_pfizer.tsv'

In [ ]:
en_hashtag_pfizer